<a href="https://colab.research.google.com/github/leiyi-ihb/ColabPractics/blob/main/Bug/loss%E8%AE%AD%E7%BB%83%E4%B8%AD%E4%B8%8D%E4%B8%8B%E9%99%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import platform
import tensorflow
import keras
from numpy.random import seed
print("Platform:{}".format(platform.platform()))
print("Tensorflow version:{}".format(tensorflow.__version__))
print("Keras version:{}".format(keras.__version__))
import numpy as np
from IPython.display import Image

from keras.models import Model
from keras.layers import Input, Dense, Conv1D, MaxPool1D
from keras.utils import plot_model
from keras.optimizers import Adam, SGD

Platform:Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Tensorflow version:2.9.2
Keras version:2.9.0


In [3]:
! if [ ! -f "dataset_Li.pkl" ]; then wget https://master.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl ; else echo "Found previous downloaded data."; fi

--2022-11-12 16:19:10--  https://master.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl
Resolving master.dl.sourceforge.net (master.dl.sourceforge.net)... 216.105.38.12
Connecting to master.dl.sourceforge.net (master.dl.sourceforge.net)|216.105.38.12|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl [following]
--2022-11-12 16:19:10--  https://downloads.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|204.68.111.105|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cfhcable.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl [following]
--2022-11-12 16:19:11--  https://cfhcable.dl.sourceforge.net/project/summer-re

In [7]:
# Data processing
import pickle
with open("dataset_Li.pkl", "rb") as f:
    data = pickle.load(f)
x_train, y_train = np.array(data['train'][0]), np.array(data['train'][1])
x_test, y_test = np.array(data['test'][0]), np.array(data['test'][1])

print(x_train.shape)

(16000, 1000, 4)


In [10]:
from keras.layers.reshaping.flatten import Flatten
from keras.layers.regularization.dropout import Dropout
class CNNModel: 
  seed(1)
  tensorflow.random.set_seed(1) 
  def __init__(self):        
    '''optimizer'''
    # self.adam = Adam(lr=0.01)
    # self.sgd = SGD(lr=0.01) 
    # self.model = Sequential()
    pass

  def modelBuild(self, filter, poolSize, kernelSize, unit):
    # model = self.model
    '''model build'''
       
    input = Input(shape=(1000, 4), name='input')
    conv1d_1 = Conv1D(filters=filter, kernel_size=kernelSize, activation='relu', name='conv1d_1')(input)
    # pool_1 = MaxPool1D(pool_size=poolSize, name='pool_1')(conv1d_1)
    conv1d_2 = Conv1D(filters=filter/2, kernel_size=kernelSize, activation='relu', name='conv1d_2')(conv1d_1)
    # pool_2 = MaxPool1D(pool_size=2, name='pool_2')(conv1d_2)
    conv1d_3 = Conv1D(filters=filter/4, kernel_size=kernelSize, activation='relu', name='conv1d_3')(conv1d_2)
    pool_3 = MaxPool1D(pool_size=2, name='pool_3')(conv1d_3)
    dense_1 = Dense(units=unit, activation='relu', name='dense1')(pool_3)
    # dense_2 = Dense(8, activation='relu', name='dense2')(dense_1)
    # drop = Dropout(0.2, name='dropout_1')(dense_1)
    flat_1 = Flatten()(dense_1)
    output = Dense(1, activation='softmax', name='output')(flat_1)

    model = Model(inputs=input, outputs=output)
    return model

  def modelCompile(self, opt, m):
    '''model compile'''
    model = m
    model.compile(optimizer=opt,
             metrics = 'acc',
             loss = 'binary_crossentropy')

  def modelFit(self, batSize, m): 
    '''model fix'''
    model = m 
    # model = self.model 
    model.fit(x_train, 
              y_train,
              epochs = 10,
              batch_size = batSize
              )
    
  def modelEvaluate(self, m):
    '''model evaluate'''
    model = m
    # model = self.model
    loss, accuracy1 = model.evaluate(x_train, y_train)
    print("\ntest loss:", loss)
    print('accuracy:', accuracy1)
    
    loss, accuracy2 = model.evaluate(x_test, y_test) #将我们的训练集也评估一下
    print("\ntrain loss:", loss)
    print('accuracy:', accuracy2)
    
    return "test_loss" + "\t" + str(accuracy1) + "\t" + "train_loss" + "\t" + str(accuracy2)


if __name__=='__main__':
  '''
  1, Conv1d: Filters = {32, 16, 3}; Kernel_size = {20, 10, 7, 5, 3}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  2, Pooling1d: Max or Avg pool; Pool size = {8, 4, 2,}
  3, Flatten: Global Max, Global Avg, Flatten
  4, Dense: Units = {32, 64}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  '''
  filter=[4, 8, 16, 32, 64, 128]  
  kernel_size =[20, 10, 7, 5, 53]
  poolSize = [8, 4, 2]
  unit=[4, 8, 16, 32]
  opt=['adam', 'sgd']
  batSize=[4, 8, 10, 12]


  from itertools import product
#   outfile = open("outFileBinaryCrossentropy.txt", "a+")
  loop_val = [filter, kernel_size, poolSize, unit, opt, batSize] #
  for i in product(*loop_val):
    # print(i)
    chr_tmp = str(i)
    if i == (4, 20, 8, 16, 'adam', 8):
      outfile = open("outFileBC.txt", "a+")
      obj1 = CNNModel()
      model = obj1.modelBuild(filter=i[0], kernelSize=i[1], poolSize=i[2], unit=i[3])
      adam =Adam(lr=0.1)
      obj1.modelCompile(opt=adam, m = model)
      print(model.summary())
      obj1.modelFit(batSize=i[5], m = model)
      scor = obj1.modelEvaluate(m=model)
    # print(scor)
      outfile.write(chr_tmp + "\t" + scor+"\n")
      outfile.close()      

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1000, 4)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 981, 4)            324       
                                                                 
 conv1d_2 (Conv1D)           (None, 962, 2)            162       
                                                                 
 conv1d_3 (Conv1D)           (None, 943, 1)            41        
                                                                 
 pool_3 (MaxPooling1D)       (None, 471, 1)            0         
                                                                 
 dense1 (Dense)              (None, 471, 16)           32        
                                                                 
 flatten_4 (Flatten)         (None, 7536)              0   

**这里总结一下loss函数训练过程中不下降的问题。当然我们这里讨论的loss不下降，默认了数据特征工程，网络结构等是没问题的，仅从参数上讨论不下降的情况，自己踩过的坑，自己填一下。**  
我在模型训练过程中发现loss不下降：  
1、踩的第一个坑是最后一层输出层，使用的激活函数是'relu'，即同隐藏层的完全一样，`老板说这是有问题的，相当于最后一层没有激活`  
2、自己修改后，发现有些情况下还是不下降，所以又查找原因，发现网上有好些这样的问题，有一个的解释是：pytorch的交叉熵（即带有crossentropy的损失函数）里自带softmax，如果在网络输出上手动加softmax就会出现loss不下降[pytorch-分类任务训练loss不变](https://blog.csdn.net/RH_Wang/article/details/110229268)【但是需要注意的这是使用的pytorch，而且针对的是多分类】  
**3、整理loss与输出层激活函数的对应关系: [keras里一般sigmoid，损失函数一般对应使用binary_crossentropy；softmax，损失函数一般对应使用categorical_crossentropy](https://blog.csdn.net/weixin_38582851/article/details/82222411)**